# Random Forest with Hyperparameter Tuning

In [ ]:
import pyspark
import sys

In [ ]:
import pyspark.sql.functions as fn

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, StringIndexer

In [ ]:
# Check spark app name
spark.sparkContext.appName

In [ ]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", True)

In [ ]:
# print runtime versions
# Python version
sys.version

In [ ]:
# Spark version
spark.version

### Exploring Data

In [ ]:
# load classification_data.csv into Spark dataframe
df = spark.read.csv('data/classification_data.csv', header=True, inferSchema=True)

In [ ]:
# check the shape of the data 
df.count(),len(df.columns)

In [ ]:
df.printSchema()

In [ ]:
# First 5 rows of Iris dataset
df.show(5)

In [ ]:
# Exploratory Data Analysis
df.describe().show()

In [ ]:
df.groupBy('label').count().show()

In [ ]:
df.groupBy('loan_purpose').count().show()

### Feature Engineering

In [ ]:
loan_purpose_indexer = StringIndexer(inputCol="loan_purpose", outputCol="loan_index").fit(df)
df = loan_purpose_indexer.transform(df)

In [ ]:
df.select(['loan_purpose','loan_index']).show(5,False)

In [ ]:
df.columns

In [ ]:
feature_cols = df.columns[2:-2]
feature_cols += ['loan_index']
feature_cols

In [ ]:
df_assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df = df_assembler.transform(df)

In [ ]:
df.printSchema()

In [ ]:
df.select(['features','label']).show(10,False)

In [ ]:
# select data for building model
model_df=df.select(['features','label'])

### Split Data - Train & Test sets

In [ ]:
# use Random Forest to train on the training set
train_df, test_df = model_df.randomSplit([0.70, 0.30], seed=42)

In [ ]:
train_df.count(), len(train_df.columns)

In [ ]:
test_df.count(), len(test_df.columns)

### Build Random Forest Model

In [ ]:
rf = RandomForestClassifier()
rf_model = rf.fit(train_df)

In [ ]:
# predict on the test set
model_predictions = rf_model.transform(test_df)

In [ ]:
# print prediction
model_predictions.show(10)

### Evaluate Model

In [ ]:
# Select (prediction, true label) to compute AUC
evaluator = BinaryClassificationEvaluator(
    labelCol='label')
rf_auc = evaluator.evaluate(model_predictions)

In [ ]:
rf_auc

In [ ]:
rf_accuracy = MulticlassClassificationEvaluator(labelCol='label',
               metricName='accuracy').evaluate(model_predictions)

In [ ]:
'The accuracy of RF on test data is {0:.0%}'.format(rf_accuracy)

### Hyperparameter Tuning

In [ ]:
evaluator = BinaryClassificationEvaluator()
rf = RandomForestClassifier()

In [ ]:
#paramGrid = (ParamGridBuilder()
#             .addGrid(rf.maxDepth, [5,10,20,25,30])
#             .addGrid(rf.maxBins, [20,30,40 ])
#             .addGrid(rf.numTrees, [5, 20,50])
#             .build())

In [ ]:
paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [10,20])
             .addGrid(rf.maxBins, [20,30])
             .addGrid(rf.numTrees, [5,20])
             .build())

In [ ]:
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, 
                    evaluator=evaluator, numFolds=5)

In [ ]:
%%time
cv_model = cv.fit(train_df)

In [ ]:
best_rf_model = cv_model.bestModel

In [ ]:
f'Best Param(maxDepth): {best_rf_model._java_obj.getMaxDepth()}'

In [ ]:
f'Best Param(maxBins): {best_rf_model._java_obj.getMaxBins()}'

In [ ]:
f'Best Param(NumTrees): {best_rf_model._java_obj.getNumTrees()}'

### Evaluate Tuned Model

In [ ]:
# Generate predictions for entire dataset
model_predictions = best_rf_model.transform(test_df)

In [ ]:
best_rf_auc = evaluator.evaluate(model_predictions)

In [ ]:
best_rf_auc

In [ ]:
true_pos=model_predictions.filter(model_predictions['label']==1).filter(model_predictions['prediction']==1).count()

In [ ]:
actual_pos=model_predictions.filter(model_predictions['label']==1).count()

In [ ]:
pred_pos=model_predictions.filter(model_predictions['prediction']==1).count()

In [ ]:
#Recall 
float(true_pos)/(actual_pos)

In [ ]:
#Precision on test Data 
float(true_pos)/(pred_pos)